### Spoken Language Processing
В этом задании предлагается обучить классификатор класса возраста по голосу (пример с тем, как это можно сделать для пола см. в семинаре)

Подумайте, как лучше предсказывать возраст (может быть разбить на группы?) и какой лосс использовать

P.S. не забудьте, что если то вы работает в Colab, то вы можете поменять среду выполнения на GPU/TPU!

Вопросы по заданию/материалам: @Nestyme

In [ ]:
!pip3 install -q timit-utils==0.9.0
!pip3 install -q torchaudio
!wget https://ndownloader.figshare.com/files/10256148 
!unzip -q 10256148

     |████████████████████████████████| 1.9MB 30.1MB/s 
--2021-05-30 19:38:55--  https://ndownloader.figshare.com/files/10256148
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 34.241.102.62, 52.212.10.43, 176.34.153.62, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|34.241.102.62|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/10256148/TIMIT.zip [following]
--2021-05-30 19:38:55--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/10256148/TIMIT.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.62.163
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.62.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 440207227 (420M) [binary/octet-stream]
Saving to: ‘10256148’

10256148            100%[===================>] 419.81M  7.35MB/s    in 26s     

2021-05-30 19:39:22 (15.9 MB/s) - 

In [ ]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

import IPython
_TIMIT_PATH = 'data/lisa/data/timit/raw/TIMIT'

In [ ]:
!nvidia-smi

Sun May 30 19:50:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Задание 1
Загрузите данные для обучения. Для этого:
1. Скачайте датасет TIMIT (см семинар)
2. Соберите пары "голос"  — "класс возраста" также, как на семинаре собирались пары "голос"  — "пол". Аудиодорожки сконвертируйте в мелспектрограммы при помощи `torchaudio либо` `librosa`

P.S. вы можете использовать свою реализацию, а можете предложенную (см следующие ячейки)

In [ ]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm import tqdm

import torch as t


class timit_dataloader:
    def __init__(self, data_path=_TIMIT_PATH, train_mode=True, age_mode=True):
        self.doc_file_path = os.path.join(data_path, 'DOC', 'SPKRINFO.TXT')
        self.corpus = tu.Corpus(data_path)
        with open(self.doc_file_path) as f:
            self.id_age_dict = dict(
                [(tmp.split(' ')[0], 86 - int(tmp.split('  ')[5].split('/')[-1].replace('??', '50'))) \
                 for tmp in f.readlines()[39:]])
        if train_mode:
            self.trainset = self.create_dataset('train', age_mode=age_mode)
            self.validset = self.create_dataset('valid', age_mode=age_mode)
        self.testset = self.create_dataset('test', age_mode=age_mode)

    def return_age(self, id):
        return self.id_age_dict[id]

    def return_data(self):
        return self.trainset, self.validset, self.testset

    def return_test(self):
        return self.testset

    def create_dataset(self, mode, age_mode=False):
        global people
        assert mode in ['train', 'valid', 'test']
        if mode == 'train':
            people = [self.corpus.train.person_by_index(i) for i in range(350)]
        if mode == 'valid':
            people = [self.corpus.train.person_by_index(i) for i in range(350, 400)]
        if mode == 'test':
            people = [self.corpus.test.person_by_index(i) for i in range(150)]
        spectrograms_and_targets = []
        for person in tqdm(people):
              try:
                  target = self.return_age(person.name)
                  for i in range(len(person.sentences)):
                      spectrograms_and_targets.append(
                          self.preprocess_sample(person.sentence_by_index(i).raw_audio, target, age_mode=True))
              except:
                  print(person.name, target)

        X, y = map(np.stack, zip(*spectrograms_and_targets))
        X = X.transpose([0, 2, 1])  # to [batch, time, channels]
        return X, y

    @staticmethod
    def spec_to_image(spec, eps=1e-6):
        mean = spec.mean()
        std = spec.std()
        spec_norm = (spec - mean) / (std + eps)
        spec_min, spec_max = spec_norm.min(), spec_norm.max()
        spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
        spec_scaled = spec_scaled.astype(np.uint8)
        return spec_scaled

    @staticmethod
    def clasterize_by_age(age):
        if age < 25:
            return 0 # 0
        if 25 < age < 40:
            return 1 # 0.5
        if age > 40:
            return 2 # 1


    # def clasterize_by_age(age):
    #     if age < 20:
    #         return 0 
    #     if 20 < age < 25:
    #         return 1
    #     if 25 < age < 30:
    #         return 2 
    #     if age > 30:
    #         return 3

    def preprocess_sample(self, amplitudes, target, age_mode=False, sr=16000, max_length=150):
        spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr, n_mels=128, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        target = self.clasterize_by_age(target)
        return self.spec_to_image(np.float32(spectrogram)), target

    def preprocess_sample_inference(self, amplitudes, sr=16000, max_length=150, device='cpu'):
        spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr, n_mels=128, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        spectrogram = np.array([self.spec_to_image(np.float32(spectrogram))]).transpose([0, 2, 1])

        return t.tensor(spectrogram, dtype=t.float).to(device, non_blocking=True)


class dataloader:
    def __init__(self, spectrograms, targets):
        self.data = list(zip(spectrograms, targets))

    def next_batch(self, batch_size, device):
        indices = np.random.randint(len(self.data), size=batch_size)

        input = [self.data[i] for i in indices]
        
        source = []
        target = []
        for line in input:
            if line[0] is not None and line[1] is not None:
                source.append(line[0])
                target.append(line[1])

        return self.torch_batch(source, target, device)

    @staticmethod
    def torch_batch(source, target, device):
        return tuple(
            [
                t.tensor(val, dtype=t.float).to(device, non_blocking=True)
                for val in [source, target] 
            ]
        )

    @staticmethod
    def padd_sequences(lines, pad_token=0):
        lengths = [len(line) for line in lines]
        max_length = max(lengths)

        return np.array(
            [
                line + [pad_token] * (max_length - lengths[i])
                for i, line in enumerate(lines)
            ]
        )

Простая сверточная сеть, ее можно дотюнить или поменять по желанию

In [ ]:
_timit_dataloader = timit_dataloader()
train, valid, test = _timit_dataloader.return_data()

trainset = dataloader(*train)
validset = dataloader(*valid)
testset = dataloader(*test)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self, window_sizes=(3, 4, 5)):
        super(Model, self).__init__()

        self.convs1 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.fc = nn.Linear(128 * len(window_sizes), 3)

    def forward(self, x):
        x = torch.unsqueeze(x, 1)  # [B, 1, T, E] Add a channel dim.
        xs = []
        for conv1 in self.convs1:

            x2 = F.relu(conv1(x)) # [B, F, T, 1]

            x2 = torch.squeeze(x2, -1)  # [B, F, T]
            x2 = F.max_pool1d(x2, x2.size(2))  # [B, F, 1]
            xs.append(x2)

        x = torch.cat(xs, 2)  # [B, F, window]

        # FC
        x = x.view(x.size(0), -1)  # [B, F * window]
        logits = self.fc(x)  # [B, class]
        probs = torch.sigmoid(logits) #.view(-1)
        return probs

    def loss(self, probs, targets):
        return nn.CrossEntropyLoss()(probs.float(), targets.long())

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using {device} mode')
patience = 500
best_loss = 1000
cnt = 0
BATCH_SIZE = 64

using cuda mode


In [ ]:
model = Model()
if device == torch.device('cuda'):
    model.cuda()
else:
    model.cpu()
model.train()

optimizer = Adam(
    [p for p in model.parameters() if p.requires_grad], betas=(0.9, 0.999), eps=1e-5
)

In [ ]:
input, target = trainset.next_batch(64, device)
out = model(input)
model.loss(out, target).item()

#Задание 2
1. Обучите свой классификатор категории возраста
2. Попробуйте улучшить результат. Можно попробовать усложнить сетку, подвигать границы категорий, поискать новые данные, что угодно, кроме учиться на тесте :)
3. Какой подход оказался самым эффективным? Как думаете, почему?
4. Как считаете, где можно было бы применить такой классификатор в качестве вспомогательной задачи?


Сначала используем Baseline модель и оценим ее качество

In [ ]:
model

Model(
  (convs1): ModuleList(
    (0): Conv2d(1, 128, kernel_size=[3, 128], stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 128, kernel_size=[5, 128], stride=(1, 1), padding=(4, 0))
  )
  (fc): Linear(in_features=384, out_features=3, bias=True)
)

In [ ]:
from tqdm.notebook  import tqdm
import torch as t


for i in tqdm(range(1500)):

    optimizer.zero_grad()

    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = model.loss(out, target)
    loss.backward()
    optimizer.step()

    if i % 50 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            valid_loss = model.loss(out, target)
            out, target = out.cpu().detach(), target.cpu().detach()
            # print(out, target)
            #out = [1 if tmp > 0.5 else 0 for tmp in out]
            # print(target)
            # print('------')
            # print(out)
            pred_class = torch.argmax(out, axis=1)
            print(f'accuracy_score:{accuracy_score(pred_class, target.long())}')
            print("i {}, valid {}".format(i, valid_loss.item()))
            print("_________")

        model.train()

    if i % 50 == 0 and best_loss > valid_loss.item():
        best_loss = valid_loss.item()
        cnt = 0
    else:
        cnt += 1

    if cnt > patience:
        break
print('training finished')

accuracy_score:0.5535714285714286
i 0, valid 0.9985033869743347
_________
accuracy_score:0.7692307692307693
i 50, valid 0.7822141051292419
_________
accuracy_score:0.6842105263157895
i 100, valid 0.8672343492507935
_________
accuracy_score:0.5862068965517241
i 150, valid 0.9652380347251892
_________
accuracy_score:0.7450980392156863
i 200, valid 0.8063468933105469
_________
accuracy_score:0.6530612244897959
i 250, valid 0.8983836770057678
_________
accuracy_score:0.625
i 300, valid 0.926444947719574
_________
accuracy_score:0.6071428571428571
i 350, valid 0.9443020820617676
_________
accuracy_score:0.6666666666666666
i 400, valid 0.8847782015800476
_________
accuracy_score:0.6326530612244898
i 450, valid 0.9187917709350586
_________
accuracy_score:0.6037735849056604
i 500, valid 0.9476713538169861
_________
accuracy_score:0.6379310344827587
i 550, valid 0.9135138392448425
_________
training finished


In [ ]:
model.eval()

with torch.no_grad():
    optimizer.zero_grad()
    acc_list = []
    for _ in range(20):
        input, target = testset.next_batch(BATCH_SIZE, device=device)
        out = model(input)
        valid_loss = model.loss(out, target)
        out, target = out.cpu().detach(), target.cpu().detach()

        pred_class = torch.argmax(out, axis=1)
        score = accuracy_score(pred_class, target.long())
        acc_list.append(score)
print('Mean accuracy score {:.3}'.format(np.mean(acc_list)))

Mean accuracy score 0.698


Это baseline результат. Теперь создадим собственную модель и улучшим результат 

In [ ]:
!nvidia-smi

Sat May 29 16:41:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   78C    P0    33W /  70W |   2458MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1) Теперь попытаемся увеличить качество, изменив количество параметров. Добавим еще три сверочных слоя. Проверим, получится ли у сетки в таком случае отискивать более сложные закономероности, что могло бы позволить улучшить качество.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class NewModel(nn.Module):
    def __init__(self, window_sizes=(3, 4, 4)):
        super(NewModel, self).__init__()

        self.convs1 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.convs2 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.convs3 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])
        self.batchnorm128 = nn.BatchNorm2d(128)
        self.batchnorm64 = nn.BatchNorm2d(64)
        self.batchnorm32 = nn.BatchNorm2d(32)
        self.fc = nn.Linear(128 * len(window_sizes), 4)

    def forward(self, x):
        x = torch.unsqueeze(x, 1)  # [B, 1, T, E] Add a channel dim.
        xs = []
        for conv1, conv2, conv3 in zip(self.convs1, self.convs2, self.convs3):

            # print(x.shape)
            x2 = F.relu(conv1(x)) # [B, F, T, 1]
            x2 = self.batchnorm128(x2)
            # print(x2.permute(0, 3, 2, 1).shape)

            x2 =  F.relu(conv2(x2.permute(0, 3, 2, 1)))
            x2 = self.batchnorm128(x2)
            # print(x2.shape)

            x2 =  F.relu(conv3(x2.permute(0, 3, 2, 1)))
            x2 = self.batchnorm128(x2)

            x2 = torch.squeeze(x2, -1)  # [B, F, T]
            x2 = F.max_pool1d(x2, x2.size(2))  # [B, F, 1]
            xs.append(x2)
            
        x = torch.cat(xs, 2)  # [B, F, window]


        # FC
        x = x.view(x.size(0), -1)  # [B, F * window]
        logits = self.fc(x)  # [B, class]
        probs = torch.sigmoid(logits) #.view(-1)
        return probs

    def loss(self, probs, targets):
        return nn.CrossEntropyLoss()(probs.float(), targets.long())

In [ ]:
model = NewModel()
if device == torch.device('cuda'):
    model.cuda()
else:
    model.cpu()
model.train()

patience = 1000
best_loss = 1000
cnt = 0
BATCH_SIZE = 64

In [ ]:
input, target = trainset.next_batch(64, device)
out = model(input)
model.loss(out, target).item()

1.3886786699295044

In [ ]:
from tqdm.notebook  import tqdm
import torch as t

optimizer = Adam(
    [p for p in model.parameters() if p.requires_grad], lr=1e-3, betas=(0.9, 0.999), eps=1e-5
)

for i in tqdm(range(500)):

    optimizer.zero_grad()

    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = model.loss(out, target)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            valid_loss = model.loss(out, target)
            out, target = out.cpu().detach(), target.cpu().detach()

            pred_class = torch.argmax(out, axis=1)
            print(f'accuracy_score:{accuracy_score(pred_class, target.long())}')
            print("i {}, valid {}".format(i, valid_loss.item()))
            print("_________")

        model.train()

    if i % 1000 == 0 and best_loss > valid_loss.item():
        best_loss = valid_loss.item()
        cnt = 0
    else:
        cnt += 1

    if cnt > patience:
        break
print('training finished')

accuracy_score:0.6938775510204082
i 0, valid 1.0943212509155273
_________
accuracy_score:0.6833333333333333
i 100, valid 1.0557618141174316
_________
accuracy_score:0.6
i 200, valid 1.1277204751968384
_________
accuracy_score:0.7659574468085106
i 300, valid 0.9621021747589111
_________
accuracy_score:0.6122448979591837
i 400, valid 1.1225416660308838
_________

training finished


In [ ]:
model.eval()

with torch.no_grad():
    optimizer.zero_grad()
    acc_list = []
    for _ in range(20):
        input, target = testset.next_batch(BATCH_SIZE, device=device)
        out = model(input)
        valid_loss = model.loss(out, target)
        out, target = out.cpu().detach(), target.cpu().detach()

        pred_class = torch.argmax(out, axis=1)
        score = accuracy_score(pred_class, target.long())
        acc_list.append(score)
print('Mean accuracy score {:.3}'.format(np.mean(acc_list)))

Mean accuracy score 0.693


Качесто осталось примеро таким же, то есть увеличение параметров не дало прирост в результатах.

2) Теперь попытаемся увеличить качество, изменив количество классов до 4. То есть мы сделаем разбиение возрастов более подробным. (для этого слегка изменим исходную функцию сосздания датасетов и и заново создадим даталоадеры)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class NewModel(nn.Module):
    def __init__(self, window_sizes=(3, 4, 4)):
        super(NewModel, self).__init__()

        self.convs1 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.convs2 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.convs3 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])
        self.batchnorm128 = nn.BatchNorm2d(128)
        self.batchnorm64 = nn.BatchNorm2d(64)
        self.batchnorm32 = nn.BatchNorm2d(32)
        self.fc = nn.Linear(128 * len(window_sizes), 4)

    def forward(self, x):
        x = torch.unsqueeze(x, 1)  # [B, 1, T, E] Add a channel dim.
        xs = []
        for conv1, conv2, conv3 in zip(self.convs1, self.convs2, self.convs3):

            # print(x.shape)
            x2 = F.relu(conv1(x)) # [B, F, T, 1]
            x2 = self.batchnorm128(x2)
            # print(x2.permute(0, 3, 2, 1).shape)

            x2 =  F.relu(conv2(x2.permute(0, 3, 2, 1)))
            x2 = self.batchnorm128(x2)
            # print(x2.shape)

            x2 =  F.relu(conv3(x2.permute(0, 3, 2, 1)))
            x2 = self.batchnorm128(x2)

            x2 = torch.squeeze(x2, -1)  # [B, F, T]
            x2 = F.max_pool1d(x2, x2.size(2))  # [B, F, 1]
            xs.append(x2)
            
        x = torch.cat(xs, 2)  # [B, F, window]


        # FC
        x = x.view(x.size(0), -1)  # [B, F * window]
        logits = self.fc(x)  # [B, class]
        probs = torch.sigmoid(logits) #.view(-1)
        return probs

    def loss(self, probs, targets):
        return nn.CrossEntropyLoss()(probs.float(), targets.long())

In [ ]:
model = NewModel()
if device == torch.device('cuda'):
    model.cuda()
else:
    model.cpu()
model.train()

optimizer = Adam(
    [p for p in model.parameters() if p.requires_grad], lr=1e-2, betas=(0.9, 0.999), eps=1e-5
)

patience = 1000
best_loss = 1000
cnt = 0
BATCH_SIZE = 64

model

NewModel(
  (convs1): ModuleList(
    (0): Conv2d(1, 128, kernel_size=[3, 128], stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
  )
  (convs2): ModuleList(
    (0): Conv2d(1, 128, kernel_size=[3, 128], stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
  )
  (convs3): ModuleList(
    (0): Conv2d(1, 128, kernel_size=[3, 128], stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
  )
  (batchnorm128): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm64): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm32): 

In [ ]:
input, target = trainset.next_batch(64, device)
out = model(input)
model.loss(out, target).item()

1.307113528251648

In [ ]:
from tqdm.notebook  import tqdm
import torch as t


for i in tqdm(range(1000)):

    optimizer.zero_grad()

    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = model.loss(out, target)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            valid_loss = model.loss(out, target)
            out, target = out.cpu().detach(), target.cpu().detach()

            pred_class = torch.argmax(out, axis=1)
            print(f'accuracy_score:{accuracy_score(pred_class, target.long())}')
            print("i {}, valid {}".format(i, valid_loss.item()))
            print("_________")

        model.train()

    if i % 1000 == 0 and best_loss > valid_loss.item():
        best_loss = valid_loss.item()
        cnt = 0
    else:
        cnt += 1

    if cnt > patience:
        break
print('training finished')

accuracy_score:0.4
i 0, valid 1.34881591796875
_________
accuracy_score:0.5576923076923077
i 100, valid 1.1859757900238037
_________
accuracy_score:0.4642857142857143
i 200, valid 1.279382348060608
_________
accuracy_score:0.4426229508196721
i 300, valid 1.3010450601577759
_________
accuracy_score:0.48214285714285715
i 400, valid 1.2615251541137695
_________
accuracy_score:0.39285714285714285
i 500, valid 1.3508108854293823
_________
accuracy_score:0.37037037037037035
i 600, valid 1.3732976913452148
_________
accuracy_score:0.45098039215686275
i 700, valid 1.2926876544952393
_________
accuracy_score:0.5
i 800, valid 1.2436679601669312
_________
accuracy_score:0.4444444444444444
i 900, valid 1.29922354221344
_________

training finished


In [ ]:
model.eval()

with torch.no_grad():
    optimizer.zero_grad()
    acc_list = []
    for _ in range(20):
        input, target = testset.next_batch(BATCH_SIZE, device=device)
        out = model(input)
        valid_loss = model.loss(out, target)
        out, target = out.cpu().detach(), target.cpu().detach()

        pred_class = torch.argmax(out, axis=1)
        score = accuracy_score(pred_class, target.long())
        acc_list.append(score)
print('Mean accuracy score {:.3}'.format(np.mean(acc_list)))

Mean accuracy score 0.447


Качество очень сильно упало, это связано с тем, что очень мало данных, и у модели не получается выучить закономерности, следовательно, она подгоняется под обучающую выборку.

Попробуем теперь сократить количество параметров и вернуться к 3 классам.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class NewModel(nn.Module):
    def __init__(self, window_sizes=(4, 5)):
        super(NewModel, self).__init__()

        self.convs1 = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.batchnorm128 = nn.BatchNorm2d(128)

        self.fc = nn.Linear(128 * len(window_sizes), 3)

    def forward(self, x):
        x = torch.unsqueeze(x, 1)  # [B, 1, T, E] Add a channel dim.
        xs = []
        for conv1 in self.convs1:

            # print(x.shape)
            x2 = F.relu(conv1(x)) # [B, F, T, 1]
            x2 = self.batchnorm128(x2)


            x2 = torch.squeeze(x2, -1)  # [B, F, T]
            x2 = F.max_pool1d(x2, x2.size(2))  # [B, F, 1]
            xs.append(x2)
            
        x = torch.cat(xs, 2)  # [B, F, window]


        # FC
        x = x.view(x.size(0), -1)  # [B, F * window]
        logits = self.fc(x)  # [B, class]
        probs = torch.sigmoid(logits) #.view(-1)
        return probs

    def loss(self, probs, targets):
        return nn.CrossEntropyLoss()(probs.float(), targets.long())

In [ ]:
patience = 1000
best_loss = 1000
cnt = 0
BATCH_SIZE = 64

model = NewModel()
if device == torch.device('cuda'):
    model.cuda()
else:
    model.cpu()
model.train()

model

NewModel(
  (convs1): ModuleList(
    (0): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
    (1): Conv2d(1, 128, kernel_size=[5, 128], stride=(1, 1), padding=(4, 0))
  )
  (batchnorm128): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=256, out_features=3, bias=True)
)

In [ ]:
from tqdm.notebook  import tqdm
import torch as t


optimizer = Adam(
    [p for p in model.parameters() if p.requires_grad], lr=1e-2, betas=(0.9, 0.999), eps=1e-5
)


for i in tqdm(range(800)):

    optimizer.zero_grad()

    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = model.loss(out, target)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            valid_loss = model.loss(out, target)
            out, target = out.cpu().detach(), target.cpu().detach()

            pred_class = torch.argmax(out, axis=1)
            print(f'accuracy_score:{accuracy_score(pred_class, target.long())}')
            print("i {}, valid {}".format(i, valid_loss.item()))
            print("_________")

        model.train()

    if i % 1000 == 0 and best_loss > valid_loss.item():
        best_loss = valid_loss.item()
        cnt = 0
    else:
        cnt += 1

    if cnt > patience:
        break
print('training finished')

accuracy_score:0.6792452830188679
i 0, valid 0.8721995949745178
_________
accuracy_score:0.660377358490566
i 100, valid 0.8910675048828125
_________
accuracy_score:0.7272727272727273
i 200, valid 0.8241721391677856
_________
accuracy_score:0.66
i 300, valid 0.8914449214935303
_________
accuracy_score:0.6862745098039216
i 400, valid 0.865170419216156
_________
accuracy_score:0.64
i 500, valid 0.9114448428153992
_________
accuracy_score:0.7241379310344828
i 600, valid 0.8273069858551025
_________
accuracy_score:0.6938775510204082
i 700, valid 0.857567310333252
_________

training finished


In [ ]:
model.eval()

with torch.no_grad():
    optimizer.zero_grad()
    acc_list = []
    for _ in range(20):
        input, target = testset.next_batch(BATCH_SIZE, device=device)
        out = model(input)
        valid_loss = model.loss(out, target)
        out, target = out.cpu().detach(), target.cpu().detach()

        pred_class = torch.argmax(out, axis=1)
        score = accuracy_score(pred_class, target.long())
        acc_list.append(score)
print('Mean accuracy score {:.3}'.format(np.mean(acc_list)))

Mean accuracy score 0.725


Нам удалось повысить качество, однако не очень значительно.

**Общий вывод**: Умньшение параметров дает прирост, но очень незначительный. Как можно видеть, на существующих данных очень сложно повысить качество. Из-за препроцессинга мы вынуждены откинуть значимую долю данных, это уменшает возможность точно натренировать сетку. Деление данных на большее количество категорий и усложнение сетки не дает прирост, тк можнль начинает переобучатся. В такой ситуации меньшее количестов параметров лучше – модель с меньшей вероятностью переобучается.

Классификация по возрасту может быть эффективна, например, в случае звонков в службу спасения или службу психологической помощи. Тогда имелась бы возможность подбирать специлиста напрямую адекватно возрасту, что повышает эффективность ответа на обращение.